Changelog
- 3 Oct
    - Creation
    - Add `Tables`, `DataValueInterfaces` and `DataAPI` because `PGFPlots` wasn't getting precompiled
    - Wrote functions for regressogram,local_averaging and Gaussian kernel regression
    - Works fine on Glass. Regressogram not working on motorcycle dataset
- 7 Oct
    - Motorcyle regressogram works; was simply a matter of name confusion between glass and motorcyle
- 8 Oct
    - All the other questions
    - Saving pgfplots
- 16 Oct
    - Hw2 local linear regression implementation
- 22 Oct
    - Hw2 part b LOOCV implementation and optimal bandwidth for motorcycle dataset
- 16 Nov
    - Homework 4: 2 layer wide neural network
- 18 Nov
    - Continuing work on hw4. Actually deploying the functions to do some compuation now

In [ ]:
# Imports
using DelimitedFiles # To use readdlm
using PGFPlots
using StatsBase # To use mean
using Distributions # To call Normal distribution
using Random # To seed

#### function definitions

In [ ]:
# function: make even bins
"""
    function make_bins(num_bins,lo,hi)
- Make `num_bins` even bins from `lo` to `hi`

# Example
```julia
bin_lefts = make_bins(10,0.29,3.50)
```
"""
function make_bins(num_bins,lo,hi;verbosity=false)
    bin_array = fill(0.,num_bins)
    if verbosity print("lo=$lo,hi=$hi,num_bins=$num_bins\n") end
    width = (hi-lo)/num_bins
    if verbosity print("width=$width\n") end
    for i in 1:num_bins
        bin_array[i] = lo+width*(i-1)
    end
    return bin_array
end

# function: Create dict with bins and corresponding average outputs
"""
    function binned_y_average(binned_y_train)
- Given a dict with bin number as keys and array of output values as value
- Return dict with same bin number keys but just the corresponding mean value

# Example
```julia
binavg_y_train = binned_y_average(binned_y_train)
```
"""
function binned_y_average(binned_y_train)
    binavg_y_train = Dict{Int64,Float64}()
    for (k,v) in binned_y_train
        binavg_y_train[k] = mean(v)
    end
    return binavg_y_train
end

In [ ]:
# function: Regressogram
"""
    function regressogram(num_bins,lo,hi,num_test,x_train,y_train)
- Divide input (covariate) range into `num_bins` from `lo` to `hi`
- Run regressogram on `num_test` evenly spaced input points in the range `lo` to `hi`
- Training is done using `x_train` and `y_train`

# How it works
- Collect training data input values into uniform bins
- Caculate corresponding average y value
- Test input is placed into bin and associated output is average computed above

# Examples
```julia
g = readdlm("glass.dat");y_train = Float64.(a[2:end,2]);x_train=Float64.(a[2:end,5]);
p_regressogram = regressogram(10,0.29,3.50,200,x_train,y_train)
```
"""
function regressogram(num_bins,lo,hi,num_test,x_train,y_train)
    bin_lefts = make_bins(num_bins,lo,hi);

    binned_y_train = Dict{Int64,Array{Float64,1}}()
    binned_x_train = Dict{Int64,Array{Float64,1}}()
    # initialize the dictionary
    for i in 1:num_bins
        binned_y_train[i] = []
        binned_x_train[i] = []
    end
    # Go over the training set and push the y values into bins to take mean later
    for (j,x_train_sample) in enumerate(x_train)
        push!(binned_y_train[argmin(abs.(x_train_sample .- bin_lefts))],y_train[j])
        push!(binned_x_train[argmin(abs.(x_train_sample .- bin_lefts))],x_train_sample)
    end
    binavg_y_train = binned_y_average(binned_y_train)
    
    x_test = make_bins(num_test,lo,hi);
    y_test = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        y_test[i] = binavg_y_train[argmin(abs.(x_test_sample .- bin_lefts))]
    end
    plot_regressogram = PGFPlots.Plots.Scatter(x_test,y_test,legendentry="Regressogram");
    return plot_regressogram
end

In [ ]:
# function: local_averaging
"""
    function local_averaging(bandwidth,lo,hi,num_test,x_train,y_train)
`lo` and `hi` are used to create `num_test` test points. They don't directly work in the algo

# How it works
- For every test input, consider the training inputs within `bandwidth` from the test input
- The associated test output is the average training outputs of these points within the bandwidth

# Examples
```julia
p_lavg = local_averaging(0.321,0.29,3.50,200,x_train,y_train)
```
"""
function local_averaging(bandwidth,lo,hi,num_test,x_train,y_train)
    x_test = make_bins(num_test,lo,hi)
    y_test_localavg = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        x_neighbors = []
        y_neighbors = []
        for (kk,x_train_sample) in enumerate(x_train)
            if abs(x_train_sample-x_test_sample) <= bandwidth
                push!(x_neighbors,x_train_sample)
                push!(y_neighbors,y_train[kk])
            end
        end
        y_test_localavg[i] = mean(y_neighbors)
    end
    plot_localavg = PGFPlots.Plots.Scatter(x_test,y_test_localavg,legendentry="Local Averaging");
    return plot_localavg
end

In [ ]:
# function: Gaussian Kernel
"""
    function gaussian_kernel(bandwidth,lo,hi,num_test,x_train,y_train)

# How it works
- For every test input, assign weights to all the training points
- These weights determine how much every training point contributes to the test output
- The weights is calculated based on the Gaussian pdf form of distance calculator

# Examples
```julia
p_gauss = gaussian_kernel(0.321,0.29,3.50,200,x_train,y_train)
```
"""
function gaussian_kernel(bandwidth,lo,hi,num_test,x_train,y_train)
    x_test = make_bins(num_test,lo,hi)
    y_test_gaussian = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        weights = fill(0.,length(x_train),)

        for (kk,x_train_sample) in enumerate(x_train)
            weights[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*((x_train_sample-x_test_sample)/bandwidth)^2)
        end
        y_test_gaussian[i] = (sum(weights.*y_train))/sum(weights)
    end
    plot_gaussian = PGFPlots.Plots.Scatter(x_test,y_test_gaussian,legendentry="Gaussian Kernel");
    return plot_gaussian
end

#### glass dataset

In [ ]:
# Compare various regressions for glass dataset
g = readdlm("glass.dat")
x_train_g=Float64.(g[2:end,5])
y_train_g = Float64.(g[2:end,2])
plot_trainingdata = PGFPlots.Plots.Scatter(x_train_g,y_train_g,legendentry="Train");
num_bins_g = 10 
lo_g=0.29
hi_g=3.50
num_test_g=200
bandwidth_g=0.321;
p_regressogram = regressogram(num_bins_g,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
p_lavg = local_averaging(bandwidth_g,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
p_gauss = gaussian_kernel(bandwidth_g,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
fig = PGFPlots.Axis([plot_trainingdata,p_regressogram,p_lavg,p_gauss],title="Glass Dataset",
    xlabel="Aluminium Content",ylabel="Refractive Index",legendPos="outer north east")
display(fig)
PGFPlots.save("figs/glass.tikz",fig)

#### motorcycle dataset

In [ ]:
# Compare approaches on motorcyle dataset
m = readdlm("motor.dat"); y_train_motor = Float64.(m[2:end,2]); x_train_motor = Float64.(m[2:end,1]);
plot_trainingdata = PGFPlots.Plots.Scatter(x_train_motor,y_train_motor,legendentry="Train");
num_bins = 20; lo_m=2.4;hi_m=57.6;num_test=200;bandwidth=2.76;
p_reg = regressogram(num_bins,lo_m,hi_m,num_test,x_train_motor,y_train_motor)
p_lavg = local_averaging(bandwidth,lo_m,hi_m,num_test,x_train_motor,y_train_motor)
p_gauss = gaussian_kernel(bandwidth,lo_m,hi_m,num_test,x_train_motor,y_train_motor)
fig_m = PGFPlots.Axis([plot_trainingdata,p_reg,p_lavg,p_gauss],title="Motorcycle Dataset",
    xlabel="Time",ylabel="Acceleration",legendPos="outer north east")
display(fig_m)
PGFPlots.save("figs/motor.tikz",fig_m)

## part b) Bias variance tradeoff

### Part 1
- $r(x) = x, \sigma= 1, n = 100$

In [ ]:
# Let's generate the synthetic data
Random.seed!(1)
x = collect(0.01:0.01:1);y = x + rand(Normal(0,1),100);
r_x = x;
p_noisy = PGFPlots.Plots.Scatter(x,y,legendentry="noisy data")
p_ground = PGFPlots.Plots.Scatter(x,r_x,legendentry="ground truth")
PGFPlots.Axis([p_noisy,p_ground],xlabel="x",ylabel="y",legendPos="outer north east")

In [ ]:
# Let's train our regressogram and local averaging and plot their output
p_regressogram = regressogram(10,0,1,200,x,r_x)
p_lavg = local_averaging(0.1,0,1,200,x,r_x)
fig_b1 = PGFPlots.Axis([p_ground,p_regressogram,p_lavg],legendPos="outer north east")
display(fig_b1)
PGFPlots.save("figs/b1.tikz",fig_b1)

### Part 2
- $r(x) = sin(2\pi x),\sigma=1,n=99$

In [ ]:
# function: Gaussian Kernel for MSE
"""
    function gaussian_kernel_for_MSE(bandwidth,lo,hi,num_test,x_train,y_train,sigma)
Returns the analytical mean squared error

# How it works
- For every test input, assign weights to all the training points
- These weights determine how much every training point contributes to the test output
- The weights is calculated based on the Gaussian pdf form of distance calculator
- sigma is the std. dev of the corrupting noise on the training data

# Examples
```julia
gaussian_kernel_for_MSE(0.1,1/99,1.,99,x,r_x,1.)
```
"""
function gaussian_kernel_for_MSE(bandwidth,lo,hi,num_test,x_train,y_train,sigma)
    x_test = make_bins(num_test,lo,hi)
    y_test_gaussian = fill(0.,length(x_test),)
    var = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        weights = fill(0.,length(x_train),)

        for (kk,x_train_sample) in enumerate(x_train)
            weights[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*((x_train_sample-x_test_sample)/bandwidth)^2)
        end
        y_test_gaussian[i] = (sum(weights.*y_train))/sum(weights)
        
        var[i] = sum(weights.^2)*sigma*sigma/(sum(weights)^2)
    end
    n = length(x_train)
    
    variance_analytical = sum(var)/n
    
    bias_analytical = sum((y_train .- y_test_gaussian).^2)/n
    
    p_true = PGFPlots.Plots.Scatter(x_train,y_train,legendentry="ground truth")
    p_analytical = PGFPlots.Plots.Scatter(x_train,y_test_gaussian,legendentry="analytical MSE")
    p_axis = PGFPlots.Axis([p_true,p_analytical],xlabel="x",ylabel="y")
#     display(p_axis)
    return bias_analytical + variance_analytical
end

In [ ]:
# Run the Gaussian kernel on noise free data for the MSE calculation
gaussian_kernel_for_MSE(0.1,1/99,1.,99,x,r_x,1.)

In [ ]:
# function: Gaussian kernel to compute sampling based MSE
"""
- Returns the square loss of the dataset
- Is subsequently used to compute the sampling based MSE by averaging the square loss over diff datasets

# Examples
```julia
gaussian_kernel_square_loss(0.1,1/99,1.,99,x,y,r_x)
```
"""
function gaussian_kernel_square_loss(bandwidth,lo,hi,num_test,x_train,y_train,y_true)
    x_test = make_bins(num_test,lo,hi)
    y_test_gaussian = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        weights = fill(0.,length(x_train),)

        for (kk,x_train_sample) in enumerate(x_train)
            weights[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*((x_train_sample-x_test_sample)/bandwidth)^2)
        end
        y_test_gaussian[i] = (sum(weights.*y_train))/sum(weights)
    end
    return sum((y_true .- y_test_gaussian).^2)/num_test
end

In [ ]:
# Perform sampling based MSE compute on 100 datasets
n = 99
x = fill(0.,n,)

sigma = 1.
for i in 1:n
    x[i] = i/n
end
r_x = sin.(2*pi*x)
seeds = collect(1:100)
square_loss = fill(0.,length(seeds),)
for seed in seeds
    Random.seed!(seed)
    y = r_x + rand(Normal(0,sigma),n)
    square_loss[seed] = gaussian_kernel_square_loss(0.1,1/99,1.,99,x,y,r_x)
end

count = 0; cumsumloss=0
expected_sq_loss = fill(0.,length(square_loss),)
for loss in square_loss
    cumsumloss += loss
    count += 1
    expected_sq_loss[count] = cumsumloss/count
end
print("Final expected_sq_loss = $(expected_sq_loss[end])\n")
fig_b2 = PGFPlots.Axis([PGFPlots.Plots.Scatter(collect(1:100),expected_sq_loss)],
    xlabel="Dataset number",ylabel="Average sampling MSE")
display(fig_b2)
PGFPlots.save("figs/b2.tikz",fig_b2)

### Part 3
- $r(x) = sin(10 \pi x),\sigma=0.1,n = 99$
- Plot for bandwidth = $0.001,0.05,0.2,100$

In [ ]:
# function: Plot variance bands around the expected estimate using Gaussian kernel
"""
# Examples
```julia
gaussian_kernel_with_variance_bands(0.05,1/n,1,n,x,r_x,sigma)
```
"""
function gaussian_kernel_with_variance_bands(bandwidth,lo,hi,num_test,x_train,y_train,sigma)
    x_test = make_bins(num_test,lo,hi)
    y_test_gaussian = fill(0.,length(x_test),)
    var = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        weights = fill(0.,length(x_train),)

        for (kk,x_train_sample) in enumerate(x_train)
            weights[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*((x_train_sample-x_test_sample)/bandwidth)^2)
        end
        y_test_gaussian[i] = (sum(weights.*y_train))/sum(weights)
        
        var[i] = sum(weights.^2)*sigma*sigma/(sum(weights)^2)
    end
    
    p_true = PGFPlots.Plots.Linear(x_train,y_train,legendentry="ground truth",style="red")
    p_expected = PGFPlots.Plots.Linear(x_train,y_test_gaussian,legendentry="exp", style="blue")
    p_add = PGFPlots.Plots.Linear(x_train,y_test_gaussian+sqrt.(var),legendentry=L"exp+\sqrt{var}",
    style="gray")
    p_subtract = PGFPlots.Plots.Linear(x_train,y_test_gaussian-sqrt.(var),legendentry=L"exp-\sqrt{var}",
    style = "yellow")
    p_noisy = PGFPlots.Plots.Scatter(x,y_train+rand(Normal(0,sigma),num_test),legendentry="noisy data")
    p_axis = PGFPlots.Axis([p_true,p_expected,p_add,p_subtract,p_noisy],
        xlabel="x",ylabel="y",title="bandwidth=$(bandwidth)",legendPos="outer north east")
    return p_axis
end

In [ ]:
n = 99;sigma=0.1
bandwidth = 100
x= fill(0.,n,)
for i in 1:n
    x[i] = i/n
end
r_x = sin.(10*pi*x)
fig_b3 = gaussian_kernel_with_variance_bands(bandwidth,1/n,1,n,x,r_x,sigma)
display(fig_b3)
PGFPlots.save("figs/b3_$(bandwidth).tikz",fig_b3)

### Part 4
- $r(x) = 0.5,\sigma=1,n=99$
- Plot for bandwidths: 0.001,0.05,0.2,100

In [ ]:
n = 99;sigma=1
bandwidth = 100
x= fill(0.,n,)
for i in 1:n
    x[i] = i/n
end
r_x = fill(0.5,length(x),)
fig_b4 = gaussian_kernel_with_variance_bands(bandwidth,1/n,1,n,x,r_x,sigma)
display(fig_b4)
PGFPlots.save("figs/b4_$(bandwidth).tikz",fig_b4)

### Part 5
- $r(x)=cos(2\pi x),\sigma=1$

In [ ]:
# function: Analytical MSE as a function of number of data points and bandwidth
"""
Compute the analytical MSE using the Gaussian kernel

# Examples
```julia
analytical_MSE(num_points,1,bandwidth)
```
"""
function analytical_MSE(n,sigma,bandwidth)
    x = fill(0.,n)
    for i in 1:n
        x[i]=i/n
    end
    r_x = cos.(2*pi*x)
    MSE_anal = gaussian_kernel_for_MSE(bandwidth,1/n,1.,n,x,r_x,sigma)
    return MSE_anal
end

In [ ]:
num_examples = [5, 20, 80, 320, 1280]
bandwidths = [0.02, 0.03, 0.048, 0.063, 0.08, 0.1, 0.12, 0.15, 0.19]
best_bandwidth = fill(0.,length(num_examples),)
for (i,num_points) in enumerate(num_examples)
    print("n = $(num_points)\n")
    MSE_anal_min = 10e6
    h_min = 100
    for bandwidth in bandwidths
#         print("h = $(bandwidth)\n")
        MSE_anal = analytical_MSE(num_points,1,bandwidth)
        if MSE_anal < MSE_anal_min
#             print("MSE_anal = $(MSE_anal)\n")
            MSE_anal_min = MSE_anal
            h_min = bandwidth
        end
    end
#     print("best_bandwidth = $(h_min)\n")
    best_bandwidth[i] = h_min
end
@show best_bandwidth
fig_5 = PGFPlots.Axis([PGFPlots.Plots.Scatter(num_examples,best_bandwidth)],
    xlabel="Number of data points",ylabel="Best bandwidth")
display(fig_5)
PGFPlots.save("figs/b5.tikz",fig_5)

# **********************Homework 2 begins***********

In [ ]:
# function: Local linear regression
"""
    function local_linear_reg(bandwidth,lo,hi,num_test,x_train,y_train)

# How it works
- Uses the Gaussian kernel
- Implements the formula given in the problem statement

# Examples
```julia
local_linear_reg(0.02,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
```
"""
function local_linear_reg(bandwidth,lo,hi,num_test,x_train,y_train)
    x_test = make_bins(num_test,lo,hi)
    y_test_local_lin_reg = fill(0.,length(x_test),)
    for (i,x_test_sample) in enumerate(x_test)
        w = fill(0.,length(x_train),)
        u = fill(0.,length(x_train),)
        
        # Compute the w and u
        for (kk,x_train_sample) in enumerate(x_train)
            u[kk] = x_train_sample-x_test_sample
            w[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*(u[kk]/bandwidth)^2)
        end
        
        # Compute the summation components within b
        b_first = 0.;b_second = 0.;
        for ii in 1:length(x_train)
            b_first += w[ii]*u[ii]*u[ii]
            b_second += w[ii]*u[ii]
        end
        
        # Compute b
        b = fill(0.,length(x_train),)
        for mm in 1:length(x_train)
            b[mm] = w[mm]*(b_first-u[mm]*b_second)
        end
        
        l = fill(0.,length(x_train),)
        for ff in 1:length(x_train)
            l[ff] = b[ff]/sum(b)
        end
        
        y_test_local_lin_reg[i] = (sum(l.*y_train))
    end
    plot_local_lin_reg = PGFPlots.Plots.Scatter(x_test,y_test_local_lin_reg,
        legendentry="h = $(bandwidth)");
    return plot_local_lin_reg
end

In [ ]:
g = readdlm("glass.dat")
x_train_g=Float64.(g[2:end,5])
y_train_g = Float64.(g[2:end,2])
plot_trainingdata = PGFPlots.Plots.Scatter(x_train_g,y_train_g,legendentry="Train");

lo_g=0.29
hi_g=3.50
num_test_g=200
bandwidth_g=0.321;

p1 = local_linear_reg(0.02,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
p2 = local_linear_reg(0.15,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
p3 = local_linear_reg(1.,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
fig = PGFPlots.Axis([plot_trainingdata,p1,p2,p3],title="Glass Dataset",
    xlabel="Aluminium Content",ylabel="Refractive Index",legendPos="outer north east")
display(fig)
PGFPlots.save("figs/hw2/a1.tikz",fig)

# Part a.2
$r(x) = x$

In [ ]:
data = readdlm("a2data.csv",',')
x_train = Float64.(data[2:end,2])
y_train = Float64.(data[2:end,3])
p_data = PGFPlots.Plots.Scatter(x_train,y_train,legendentry="data")
p_local_lin_reg = local_linear_reg(0.46,0.,1.,200,x_train,y_train)
p_gaussian_kernel= gaussian_kernel(0.05,0.,1.,200,x_train,y_train)
p_truth = PGFPlots.Plots.Scatter(collect(0:0.01:1),collect(0:0.01:1),legendentry="true function")
a2 = PGFPlots.Axis([p_data,p_local_lin_reg,p_gaussian_kernel,p_truth],
    xlabel="x",ylabel="y",legendPos="outer north east",title="Synthetic Data: Local linear vs Kernel estimator")
display(a2)
PGFPlots.save("figs/hw2/a2.tikz",a2)

In [ ]:
# Cleaning up glass data i.e. leave only what's required for python to load in to run splines
g = readdlm("glass.dat")
x_train_g=Float64.(g[2:end,5])
y_train_g = Float64.(g[2:end,2])

x_train = reshape(x_train_g,length(x_train_g),1)
y_train = reshape(y_train_g,length(y_train_g),1)
glass_cleaned = hcat(x_train,y_train)


In [ ]:
open("glass_cleaned.txt", "w") do io
    writedlm(io,glass_cleaned)
end

## Part C.2
$r(x) = cos(2\pi x)$ and $r(x)=x$

In [ ]:
function c2(x)
    if 0<=x && x<1
        return cos(12*pi*x)
    end
    if 1 <= x && x<=2
        return x
    end
end

In [ ]:
x_train = make_bins(200,0.,2.)
y_train = c2.(x_train)
p_true = PGFPlots.Plots.Scatter(x_train,y_train,legendentry="true")

In [ ]:
p_local_lin_reg = local_linear_reg(0.015,0.,2.,200,data_provided[:,1],data_provided[:,2]);

In [ ]:
spline_data = reshape(readdlm("spline.txt"),200,)
p_spline= PGFPlots.Plots.Scatter(x_train,spline_data,legendentry="spline")

In [ ]:
data_provided = readdlm("c2_provided_data.txt")
p_provided_data = PGFPlots.Plots.Scatter(data_provided[:,1],data_provided[:,2],legendentry="data")

In [ ]:
p_c2 = PGFPlots.Axis([p_true,p_provided_data,p_spline,p_local_lin_reg],
    xlabel="x",ylabel="y",title="Synthetic data: Spline vs Local Linear Regression",legendPos="outer north east")
display(p_c2)
PGFPlots.save("figs/hw2/p_c2.tikz",p_c2)

## problem b: Cross validation

In [ ]:
# function: Local linear regression
"""
    function loocv_local_linear_reg(bandwidth,lo,hi,x_train,y_train)
- Computes the leave one out cross validation score

# How it works
- Core code is same as the local linear regression
- Implements the formula given in the problem statement

# Examples
```julia
local_linear_reg(0.02,lo_g,hi_g,num_test_g,x_train_g,y_train_g)
```
"""
function loocv_local_linear_reg(bandwidth,lo,hi,x_train,y_train)
    n = length(x_train)
    running_sum = 0
    for (i,train_sample) in enumerate(x_train)
        w = fill(0.,length(x_train),)
        u = fill(0.,length(x_train),)
        
        # Compute the w and u
        for (kk,x_train_sample) in enumerate(x_train)
            u[kk] = x_train_sample-train_sample
            w[kk] = (1/(sqrt(2*3.14)))*exp((-1/2)*(u[kk]/bandwidth)^2)
        end
        
        # Compute the summation components within b
        b_first = 0.;b_second = 0.;
        for ii in 1:length(x_train)
            b_first += w[ii]*u[ii]*u[ii]
            b_second += w[ii]*u[ii]
        end
        
        # Compute b
        b = fill(0.,length(x_train),)
        for mm in 1:length(x_train)
            b[mm] = w[mm]*(b_first-u[mm]*b_second)
        end
        
        l = fill(0.,length(x_train),)
        for ff in 1:length(x_train)
            l[ff] = b[ff]/sum(b)
        end
        
        rhat_xi = (sum(l.*y_train))
        
        running_sum += ((y_train[i] - rhat_xi)/(1-l[i]))^2
    end
    return running_sum/n
end

In [ ]:
# Script: Variatoin of LOOCV with bandwidth
m = readdlm("motor.dat"); y_train = Float64.(m[2:end,2]); x_train = Float64.(m[2:end,1]);
bandwidths = collect(150:1:200)*0.01
Rhat_h = fill(0.,length(bandwidths),)
lo=2.4
hi=57.6
for (i,h) in enumerate(bandwidths)
    Rhat_h[i] = loocv_local_linear_reg(h,lo,hi,x_train,y_train)
end
p = PGFPlots.Axis(PGFPlots.Plots.Scatter(bandwidths,Rhat_h),
    title="LOOCV vs bandwidth for motor dataset",xlabel="h",ylabel=L"\hat{R}_h")
display(p)
PGFPlots.save("figs/hw2/b1.tikz",p)

In [ ]:
# Optimal bandwidth performance
h_opt = bandwidths[argmin(Rhat_h)]
plot_trainingdata = PGFPlots.Plots.Scatter(x_train,y_train,legendentry="Data")
plot_local_lin_reg = local_linear_reg(h_opt,lo,hi,200,x_train,y_train)
b2 = PGFPlots.Axis([plot_trainingdata,plot_local_lin_reg],xlabel="x",ylabel="y",
    title="Local linear regression with optimal bandwidth for motor dataset",legendPos="outer north east")
display(b2)
PGFPlots.save("figs/hw2/b2.tikz",b2)

# **Homework 4: 2 layer, wide neural network **

In [ ]:
# Read in the data from provided data file
data = readdlm("datahw4.csv",',')
x = vec(data[:,1])
y = vec(data[:,2]);

In [ ]:
# functions: indicator and relu
function relu(x)
    return max(0,x)
end

function indicator(x)
    if x<0
        return 0
    else
        return 1
    end
end

In [ ]:
# function: the neural net
"""
- The neural network
- a,b,w are vectors of length m initialized above and used globally by all functions
"""
function f(x,a,b,w)
    return sum(a.*relu.(w*x+b))
end

In [ ]:
# function: neural net compute on a vector input rather than scalar input
"""
Deploy neural net on vector of inputs rather than one input x

# Examples
```julia
f_xvector(x,a,b,w)
```
"""
function f_xvector(x,a,b,w)
    n=length(x)
    f_vec = vec(fill(0.,n))
    for i in 1:n
        f_vec[i] = f(x[i],a,b,w)
    end
    return f_vec
end

In [ ]:
# function: initialize parameters
"""
- Initialize the neural net parameters using a normal distribution wih `stddev`

# Examples
```julia
a,b,w = init_params(10,1.0)
```
"""
function init_params(m,stddev)
    init_dist = Normal(0,stddev)
    a = vec(rand(init_dist,m))
    b = vec(rand(init_dist,m))
    w = vec(rand(init_dist,m))
    return a,b,w
end

In [ ]:
# function: jth element of gradients computation
"""
- Compute element `j` of gradient with respect to bias parameter `b`
- `b` is a vector of length `m`. This function is returning the jth element of the gradient vector
- `x` and `y` are training data of length `n`

# Examples
```julia
# Compute each 5th element for gradient with respect to a,w and b respectively
x = vec([1 2 3])
y = vec([4 5 6])
a = vec([6 3 0 8 6 2])
b = vec([9 8 1 2 8 0])
w = vec([5 3 0 2 1 1])
compute_grad_j(x,y,0.1,5)
# Expected answer: (2671.933333333333, 3404.1, 1578.0)
```
"""

function compute_grad_j(x,y,lambda,j,a,b,w)
    n = length(x)
    grad_a = (1/n)*sum((f_xvector(x,a,b,w)-y).*relu.(w[j]*x.+b[j])) + lambda*a[j]
    grad_w = (1/n)*sum((f_xvector(x,a,b,w)-y).*a[j].*indicator.(w[j]*x.+b[j]).*x) + lambda*w[j]
    grad_b = (1/n)*sum((f_xvector(x,a,b,w)-y).*a[j].*indicator.(w[j]*x.+b[j]))
    
    return grad_a,grad_w,grad_b
end

In [ ]:
# function: one step of gradient descent
"""
- Performs one step of gradient descent and returns updated neural net parameters
"""
function update_params(x,y,eta,a,b,w,lambda)
    m = length(a)
    gradvec_a = fill(0.,m,)
    gradvec_w = fill(0.,m)
    gradvec_b = fill(0.,m)
    for j in 1:m
        gradvec_a[j],gradvec_w[j],gradvec_b[j] = compute_grad_j(x,y,lambda,j,a,b,w)
    end
    return a.-eta.*gradvec_a, w.-eta.*gradvec_w, b.-eta.*gradvec_b
end

In [ ]:
# function: T steps of gradient descent
"""
- Runs `T` iterations of gradient descent and returns final parameter values `a`,`b`,`w`
"""
function iterate(x,y,eta,a,b,w,T,lambda)
    for iternum in 1:T
        print("iterate says: iternum = $(iternum)")
        a,b,w = update_params(x,y,eta,a,b,w,lambda)
    end
    return a,b,w
end

In [ ]:
# part b: m = 10
a,b,w = init_params(10,1.0)
eta = 0.01
lambda = 0.0
numiter = 1e5
at,bt,wt = iterate(x,y,eta,a,b,w,numiter,lambda)
xtest = range(0,stop=1,length=400)
ytest = vec(fill(0.0,length(xtest)))
for i in 1:length(ytest)
    ytest[i] = f(xtest[i],at,bt,wt)
end
p_train = PGFPlots.Plots.Scatter(x,y,legendentry = "training data")
p_test = PGFPlots.Plots.Scatter(xtest,ytest,legendentry="test")
p = PGFPlots.Axis([p_train,p_test],xlabel="x",ylabel="y",title="training data and fit function")

In [ ]:
# part c: m = 200,stdev=2.0,eta = 0.001,numiter=1e5
a,b,w = init_params(200,2.0)
eta = 0.001
lambda = 0.0
numiter = 1e5
at,bt,wt = iterate(x,y,eta,a,b,w,numiter,lambda)
xtest = range(0,stop=1,length=400)
ytest = vec(fill(0.0,length(xtest)))
for i in 1:length(ytest)
    ytest[i] = f(xtest[i],at,bt,wt)
end
p_train = PGFPlots.Plots.Scatter(x,y,legendentry = "training data")
p_test = PGFPlots.Plots.Scatter(xtest,ytest,legendentry="test")
p = PGFPlots.Axis([p_train,p_test],xlabel="x",ylabel="y",title="training data and fit function")